In [2]:
%pip install langchain_groq

  Using cached charset_normalizer-3.3.2-cp310-cp310-win_amd64.whl.metadata (34 kB)
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/1.9 MB 1.8 MB/s eta 0:00:01
   --------------------------- ------------ 1.3/1.9 MB 2.0 MB/s eta 0:00:01
   -------------------------------------- - 1.8/1.9 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 2.0 MB/s eta 0:00:00
Using cached charset_normalizer-3.3.2-cp310-cp310-win_amd64.whl (100 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
api_key = "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"

**Q1)**

**Zero shot prompting**

In [5]:
import numpy as np
from langchain_groq.chat_models import ChatGroq
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import re
import matplotlib.pyplot as plt
from HAR.MakeDataset import X_train,X_test,y_train,y_test



label_mapping = {
    "LAYING": 1,
    "SITTING": 2,
    "STANDING": 3,
    "WALKING": 4,
    "WALKING_DOWNWARDS": 5,
    "WALKING_UPWARDS": 6
}
model_name = "llama3.1-8b"
model = ChatGroq(api_key=api_key)
# Function to format a single example for zero-shot learning
def format_example(example):
    ax = example[:, 0]
    ay = example[:, 1]
    az = example[:, 2]

    ax_str = ", ".join(map(str, ax))
    ay_str = ", ".join(map(str, ay))
    az_str = ", ".join(map(str, az))

    # Simplified prompt
    formatted_string = (
        f"Classify the activity based on the following accelerometer data:\n"
        f"ax: [{ax_str}]\n"
        f"ay: [{ay_str}]\n"
        f"az: [{az_str}]\n"
        f"Activity: "
    )

    return formatted_string

# Prepare input examples from x_test
formatted_examples = [format_example(example) for example in X_test]

# Perform zero-shot prediction and print raw predictions
y_pred = [model.predict(example) for example in formatted_examples]

# Extract activity labels from the model's response
def extract_label(response):
    match = re.search(r'\b(LAYING|SITTING|STANDING|WALKING|WALKING_DOWNWARDS|WALKING_UPWARDS)\b', response.upper())
    return match.group(0) if match else "UNKNOWN"

# Convert predictions to numeric labels
y_pred_labels = [label_mapping.get(extract_label(pred), 0) for pred in y_pred]

# Print raw predictions, predicted labels, and actual labels for the first 10 samples
#print("Example\tRaw Prediction\tPredicted Label\tActual Label")
for i in range(5):
    raw_pred = y_pred[i]
    pred_label = label_mapping.get(extract_label(raw_pred), 0)
    actual_label = y_test[i]
    #print(f"{i+1}\t{raw_pred}\t{pred_label}\t{actual_label}")

# Print the numeric predictions for all samples
print("\nAll predicted labels:")
print(y_pred_labels)
print("\n All True lables:")
print(y_test)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"\nAccuracy: {accuracy}")


Training data shape:  (126, 500, 3)
Testing data shape:  (54, 500, 3)


c:\Users\Shiva\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(



All predicted labels:
[4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 0, 4, 2, 0, 4, 4, 4, 4, 4, 2, 4, 3, 4, 4, 4, 4, 4, 3, 0, 4, 0, 4, 0, 0, 3, 4, 3, 0, 2, 2, 2, 4, 4, 4, 4, 4, 4]

 All True lables:
[3 1 2 5 5 1 1 5 3 2 6 5 6 5 6 1 6 5 2 5 4 3 2 2 1 4 6 4 1 2 6 2 4 4 3 6 6
 3 1 5 3 2 1 4 4 4 5 1 3 3 3 6 2 4]

Accuracy: 0.1111111111111111


**2) Few shot prompting**

In [9]:
import numpy as np
import random
from langchain_groq.chat_models import ChatGroq
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import re
import matplotlib.pyplot as plt
from HAR.MakeDataset import X_train,X_test,y_train,y_test

# Load data


# Label mappings
label_mapping = {
    "LAYING": 1,
    "SITTING": 2,
    "STANDING": 3,
    "WALKING": 4,
    "WALKING_DOWNWARDS": 5,
    "WALKING_UPWARDS": 6
}
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# Function to format a single example for few-shot learning
def format_example(example):
    ax = example[:, 0][:5]  # Increase data points used (5 for more context)
    ay = example[:, 1][:5]
    az = example[:, 2][:5]

    formatted_string = (
        f"ax: {ax}\n"
        f"ay: {ay}\n"
        f"az: {az}\n"
        f"Activity: "
    )

    return formatted_string

# Prepare few-shot examples with 4 examples per label
few_shot_examples = []
few_shot_labels = []

for label_name, label_id in label_mapping.items():
    # Filter examples by label
    label_indices = np.where(y_train == label_id)[0]
    selected_indices = random.sample(list(label_indices), 4)  # 4 random examples

    for idx in selected_indices:
        example = X_train[idx]
        indices = np.random.choice(example.shape[0], 5, replace=False)  # 5 data points
        few_shot_examples.append(example[indices])
        few_shot_labels.append(label_id)

# Format the few-shot examples with their labels
few_shot_prompts = [f"{format_example(ex)} {reverse_label_mapping[label_id]}" for ex, label_id in zip(few_shot_examples, few_shot_labels)]

# Format input examples from x_test
formatted_examples = [format_example(example) for example in X_test]

# Combine few-shot prompts with each test example for prediction
def create_few_shot_prompt(test_example):
    few_shot_section = "\n\n".join(few_shot_prompts)
    return f"{few_shot_section}\n\n{test_example}"

# Initialize the model with the correct model ID
model_name = "llama-3.1-8b-instant"
model = ChatGroq(model_name=model_name, api_key=api_key)

# Perform predictions using the ChatGroq model
def predict_with_groq(formatted_examples, limit=6):
    y_pred = []
    for i in range(0, len(formatted_examples), limit):
        batch_examples = formatted_examples[i:i+limit]
        for example in batch_examples:
            # Call the model to get a prediction
            response = model.predict(create_few_shot_prompt(example))
            y_pred.append(response)
    return y_pred

# Get predictions for all test examples using the ChatGroq model
y_pred = predict_with_groq(formatted_examples, limit=6)

# Extract activity labels from the model's response
def extract_label(response):
    match = re.search(r'\b(LAYING|SITTING|STANDING|WALKING|WALKING_DOWNWARDS|WALKING_UPWARDS)\b', response.upper())
    return match.group(0) if match else "UNKNOWN"

# Convert predictions to numeric labels
y_pred_labels = [label_mapping.get(extract_label(pred), 0) for pred in y_pred]

# Print the numeric predictions for all samples
print("All predicted labels:")
print(y_pred_labels)

print("All True labels:")
print(y_test)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"Accuracy: {accuracy}")


All predicted labels:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
All True labels:
[3 1 2 5 5 1 1 5 3 2 6 5 6 5 6 1 6 5 2 5 4 3 2 2 1 4 6 4 1 2 6 2 4 4 3 6 6
 3 1 5 3 2 1 4 4 4 5 1 3 3 3 6 2 4]
Accuracy: 0.16666666666666666


**Q2)**

Decision trees are way more accurate than few-shot prompting, as we custom make a model which works for the accelerometer data, but few-shot prompting matches the examples instead of learning with LLM.

Moreover we cannot use all the data as examples for few shot, as we encounter rate limiting errors. But in case of decision trees, we can actually use whole data without any such problems.

Also we see latency for API calling in case of few shot, which slows the whole process, though we gave fewer data. But no such thing exists for decision trees.

**Q3)**

Firstly, both of them use an LLM to predict the accelerometer data, on which no existing LLM is trained as of now. Then we want to predict the data labels using that, so it will not give good accuracies.

Also the data loses its value. We know that data is normalized in only [-1,1]. So precision in decimal points hold a significant value, which will be lost in both of the cases.

We can hence see why the accuracies are so low for both of them. It doesn't learn anything. In few shot, it creates a bias on where to map the testing data. So, These are all limitations of zero and few shot learning.

**Q4)**

Well, when new data is given, the zero-shot still plays mapping to a known existing class. As there are no examples to have bias. Where as few-shot which is expected to give better results as it has examples of it. However, if there is mis-match between training and testing data for few-shot mapping, it will bluntly map to whetever the data is nearest to.

**Q5)**

In [11]:
import numpy as np

# Set dimensions
train_size = 126
test_size = 54
timesteps = 500
features = 3

# Generate random data normalized to [-1, 1]
x_train = np.random.uniform(-1, 1, size=(train_size, timesteps, features))
x_test = np.random.uniform(-1, 1, size=(test_size, timesteps, features))

# Generate random labels for y_train and y_test within the range of your existing labels (1 to 6)
y_train = np.random.randint(1, 7, size=(train_size,))
y_test = np.random.randint(1, 7, size=(test_size,))

# Save the generated data (optional)
np.save('Task_3 Random Datasets/X_train_random.npy', x_train)
np.save('Task_3 Random Datasets/Y_train_random.npy', y_train)
np.save('Task_3 Random Datasets/X_test_random.npy', x_test)
np.save('Task_3 Random Datasets/Y_test_random.npy', y_test)

print("Random data generated.")


Random data generated.


Zero shot:

In [12]:
# Load the unseen data instead of previously loaded data
x_test = np.load('Task_3 Random Datasets/X_test_random.npy')  # Newly generated unseen data
y_test = np.load('Task_3 Random Datasets/Y_test_random.npy')  # Newly generated unseen labels

# The rest of the code remains unchanged
# Perform zero-shot prediction and print raw predictions
y_pred = [model.predict(example) for example in formatted_examples]

# Extract activity labels from the model's response
def extract_label(response):
    match = re.search(r'\b(LAYING|SITTING|STANDING|WALKING|WALKING_DOWNWARDS|WALKING_UPWARDS)\b', response.upper())
    return match.group(0) if match else "UNKNOWN"

# Convert predictions to numeric labels
y_pred_labels = [label_mapping.get(extract_label(pred), 0) for pred in y_pred]

# Print raw predictions, predicted labels, and actual labels for the first 10 samples
for i in range(5):
    raw_pred = y_pred[i]
    pred_label = label_mapping.get(extract_label(raw_pred), 0)
    actual_label = y_test[i]
    print(f"Sample {i+1} - Prediction: {pred_label}, Actual: {actual_label}")

# Print the numeric predictions for all samples
print("\nAll predicted labels:")
print(y_pred_labels)
print("\nAll True labels:")
print(y_test)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"\nAccuracy: {accuracy}")


Sample 1 - Prediction: 0, Actual: 2
Sample 2 - Prediction: 0, Actual: 2
Sample 3 - Prediction: 2, Actual: 2
Sample 4 - Prediction: 0, Actual: 3
Sample 5 - Prediction: 0, Actual: 2

All predicted labels:
[0, 0, 2, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 3, 4, 0, 0, 4, 0, 0, 4, 4, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0]

All True labels:
[2 2 2 3 2 5 3 4 1 6 4 5 2 6 1 5 4 1 4 6 6 6 5 3 1 5 1 5 6 4 5 1 3 1 1 2 6
 6 2 2 3 4 1 2 6 6 1 6 1 5 6 1 3 3]

Accuracy: 0.037037037037037035


Few shot:

In [13]:
import numpy as np
import random
from langchain_groq.chat_models import ChatGroq
from sklearn.metrics import accuracy_score
import re
from HAR.MakeDataset import X_train,X_test,y_train,y_test

# Label mappings
label_mapping = {
    "LAYING": 1,
    "SITTING": 2,
    "STANDING": 3,
    "WALKING": 4,
    "WALKING_DOWNWARDS": 5,
    "WALKING_UPWARDS": 6
}
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# Function to format a single example for few-shot learning
def format_example(example):
    ax = example[:, 0][:5]  # Increase data points used (5 for more context)
    ay = example[:, 1][:5]
    az = example[:, 2][:5]

    formatted_string = (
        f"ax: {ax}\n"
        f"ay: {ay}\n"
        f"az: {az}\n"
        f"Activity: "
    )

    return formatted_string

# Prepare few-shot examples with 4 examples per label
few_shot_examples = []
few_shot_labels = []

for label_name, label_id in label_mapping.items():
    # Filter examples by label
    label_indices = np.where(y_train == label_id)[0]
    selected_indices = random.sample(list(label_indices), 4)  # 4 random examples

    for idx in selected_indices:
        example = x_train[idx]
        indices = np.random.choice(example.shape[0], 5, replace=False)  # 5 data points
        few_shot_examples.append(example[indices])
        few_shot_labels.append(label_id)

# Format the few-shot examples with their labels
few_shot_prompts = [f"{format_example(ex)} {reverse_label_mapping[label_id]}" for ex, label_id in zip(few_shot_examples, few_shot_labels)]

# Format input examples from x_test
formatted_examples = [format_example(example) for example in x_test]

# Combine few-shot prompts with each test example for prediction
def create_few_shot_prompt(test_example):
    few_shot_section = "\n\n".join(few_shot_prompts)
    return f"{few_shot_section}\n\n{test_example}"

# Initialize the model with the correct model ID
model_name = "llama-3.1-8b-instant"
model = ChatGroq(model_name=model_name, api_key=api_key)

# Perform predictions using the ChatGroq model
def predict_with_groq(formatted_examples, limit=6):
    y_pred = []
    for i in range(0, len(formatted_examples), limit):
        batch_examples = formatted_examples[i:i+limit]
        for example in batch_examples:
            # Call the model to get a prediction
            response = model.predict(create_few_shot_prompt(example))
            y_pred.append(response)
    return y_pred

# Get predictions for all test examples using the ChatGroq model
y_pred = predict_with_groq(formatted_examples, limit=6)

# Extract activity labels from the model's response
def extract_label(response):
    match = re.search(r'\b(LAYING|SITTING|STANDING|WALKING|WALKING_DOWNWARDS|WALKING_UPWARDS)\b', response.upper())
    return match.group(0) if match else "UNKNOWN"

# Convert predictions to numeric labels
y_pred_labels = [label_mapping.get(extract_label(pred), 0) for pred in y_pred]

# Print the numeric predictions for all samples
print("All predicted labels:")
print(y_pred_labels)

print("All True labels:")
print(y_test)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"Accuracy: {accuracy}")


All predicted labels:
[1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
All True labels:
[3 1 2 5 5 1 1 5 3 2 6 5 6 5 6 1 6 5 2 5 4 3 2 2 1 4 6 4 1 2 6 2 4 4 3 6 6
 3 1 5 3 2 1 4 4 4 5 1 3 3 3 6 2 4]
Accuracy: 0.12962962962962962
